In [315]:
import pandas as pd

In [316]:
def prepare(data):
    data['import.start'] = pd.to_datetime(data['import.start'])
    data['import.stop'] = pd.to_datetime(data['import.stop'])
    data['created'] = pd.to_datetime(data['created'])
    data['started'] = pd.to_datetime(data['started'])
    data['finished'] = pd.to_datetime(data['finished'])
    
    data['starttime'] = (data['started'] - data['created']).dt.total_seconds() 
    data['dockertime'] = (data['finished'] - data['started']).dt.total_seconds() 
    data['importtime'] = (data['import.stop'] - data['import.start']).dt.total_seconds() 
    
    started_temp = pd.DataFrame()
    docker_temp = pd.DataFrame()
    import_temp = pd.DataFrame()
    
    for i in range(8,29,5):
        started_temp.insert(range(8,29,5).index(i), "", (pd.to_datetime(data.iloc[:, i+1]) - pd.to_datetime(data.iloc[:, i])).dt.total_seconds() , True)
        docker_temp.insert(range(8,29,5).index(i), "", (pd.to_datetime(data.iloc[:, i+2]) - pd.to_datetime(data.iloc[:, i+1])).dt.total_seconds() , True)
        import_temp.insert(range(8,29,5).index(i), "", (pd.to_datetime(data.iloc[:, i+4]) - pd.to_datetime(data.iloc[:, i+3])).dt.total_seconds() , True)
        
    
    started_temp.columns = ['starttime{}'.format(i) for i in range(1, 6)]
    docker_temp.columns = ['dockertime{}'.format(i) for i in range(1, 6)]
    import_temp.columns = ['importtime{}'.format(i) for i in range(1, 6)]
    
    data = pd.concat([data, started_temp, docker_temp, import_temp], axis=1)
    
    return data

In [317]:
def extract_imports(imp):
    imp = imp.split("'")
    l = ""
    
    for i in imp:
        if i == "(" or i == ")" or i == ",)" or i == "," or i == "()" or i == ", ":
            continue
        else:
            if l == "":
                l = i
            else:
                l = l + ";" + i
    return l

This is one experiment run with using the top 10 packages over all functions
The installationtime is set to minus -1 if there was no install as for native python packages as os

In [318]:
data = pd.read_csv("data/all_top_10/run01.csv")
data = prepare(data)
data = data.loc[:, ['imports', 'buldingtime', 'installtime', 'starttime', 'dockertime',
       'importtime', 'starttime1', 'starttime2', 'starttime3', 'starttime4',
       'starttime5', 'dockertime1', 'dockertime2', 'dockertime3',
       'dockertime4', 'dockertime5', 'importtime1', 'importtime2',
       'importtime3', 'importtime4', 'importtime5']]

In [319]:
data.head()

,imports,buldingtime,installtime,starttime,dockertime,importtime,starttime1,starttime2,starttime3,starttime4,...,dockertime1,dockertime2,dockertime3,dockertime4,dockertime5,importtime1,importtime2,importtime3,importtime4,importtime5
0,argparse;copy;datetime;funcx;itertools;json;ma...,33.8,29.6,2.705087,4.425136,3.014702,1.150644,0.951442,1.021779,1.218273,...,1.577872,1.566395,1.552404,1.536692,1.550373,0.469661,0.458105,0.443868,0.415803,0.444357
1,argparse;copy;funcx;itertools;json;matplotlib;...,29.9,26.5,3.753590,5.792128,4.418994,0.848623,0.924125,0.801321,0.927544,...,1.549871,1.546753,1.544053,1.555687,1.581340,0.443171,0.438238,0.441210,0.449839,0.471913
2,argparse;sys,10.2,7.7,2.903107,1.640716,0.120066,1.035499,1.041559,1.145632,0.959571,...,1.035915,1.034411,1.036701,1.033653,1.036441,0.004059,0.002662,0.006202,0.007091,0.007281
3,CLEAN,9.9,6.7,2.867039,1.637754,0.005848,0.843502,1.564489,1.442526,1.143089,...,1.029524,1.026911,1.027171,1.028132,1.026027,0.000016,0.000025,0.000028,0.000013,0.000023
4,CLEAN;os,8.7,5.6,2.583785,1.664388,0.006076,0.856858,1.144579,1.034260,0.925805,...,1.027819,1.029581,1.027125,1.027228,1.028558,0.000336,0.000139,0.000020,0.000011,0.000025


This is the mapping between imports and function
For this merge and the following ones, there is no need to think about on which join-operation. I cleaned the data beforehand. That is, there will be always a pair 
```python
pd.merge(func_mapping,data, on='imports')
```

In [320]:
func_mapping = pd.read_csv('experiments/function_mapping.csv')
func_mapping = func_mapping.drop(2)
func_mapping['func4.imports'] = func_mapping[func_mapping['func4.imports'].str.len() >= 0]['func4.imports'].map(lambda x: extract_imports(x))
func_mapping = func_mapping[func_mapping['func4.imports'] != ""]
func_mapping.rename(columns={'func4.imports': 'imports'}, inplace=True)
func_mapping.drop(columns=['Unnamed: 0'], inplace=True)

In [321]:
func_mapping.head()

,func4.name,imports,Cluster
0,0-04-_check_slurm_job_statuses.py,subprocess,Job
3,873-04-call_mtrack.py,mtrack;sys,Data: mtrack
4,43-04-make_corr_plots.py,json;os;xpcs_webplot,Sci: Instruments
6,3-04-submit_slurm_job.py,os;subprocess,Job
7,397-01-rejection_sample.py,random,Trivial


Here is the mapping between functions and task
```python
pd.merge(func_mapping,task_mapping, on='func4.name')
```

In [322]:
task_mapping = pd.read_csv("funcx/task_function_mapping.csv")

In [323]:
task_mapping.head()

,task_uuid,func4.name,executiontime
0,000001d5-b3f0-4c17-8074-04f013f7bf52,0-04-_check_slurm_job_statuses.py,0.0243
1,00002521-04ba-4fb6-9c38-92ba23dc00d7,0-04-_check_slurm_job_statuses.py,0.0331
2,00002642-f850-42b4-9e1e-2698c3396498,0-04-_check_slurm_job_statuses.py,0.0432
3,00002deb-cf79-43ed-8aff-3281938809fe,0-04-_check_slurm_job_statuses.py,0.0268
4,0000754b-898e-43dc-9b55-35d1a6fe6633,0-04-_check_slurm_job_statuses.py,0.0284


Here we have the timestamp for each task
For merging
```python
pd.merge(task_mapping,task_mapping, on='task_uuid')
```

In [324]:
task = pd.read_csv("funcx/tasks_pt1.csv")
task = pd.concat([task, pd.read_csv("funcx/tasks_pt2.csv")], ignore_index=True)
task['received'] = task['received'].interpolate()

In [325]:
task.head()

,task_uuid,user_id,endpoint_uuid,received
0,000001d5-b3f0-4c17-8074-04f013f7bf52,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.681417e+18
1,00002521-04ba-4fb6-9c38-92ba23dc00d7,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.679386e+18
2,00002642-f850-42b4-9e1e-2698c3396498,286.0,cbc1f8d4-71b2-4a41-8ed4-9c75421eb89f,1.683246e+18
3,00002deb-cf79-43ed-8aff-3281938809fe,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.677897e+18
4,0000754b-898e-43dc-9b55-35d1a6fe6633,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.679277e+18


Here are information about the endpoints
For merging:
```python
pd.merge(task,endpoints, on='endpoint_uuid')
```

In [326]:
endpoints = pd.read_csv("funcx/endpoints.csv")

In [327]:
endpoints.head()

,endpoint_uuid,X_type,ip,cluster
0,00000000-0000-4000-8000-000011110000,LocalProvider,216.80.70.252,6
1,001fecc2-1393-4bd9-950e-5645e15e3a53,SlurmProvider,141.142.140.194,10
2,006f558d-df82-45c2-b2d5-94274ef41a69,SlurmProvider,141.142.140.194,10
3,00824163-569c-4f0d-953d-097cd3b7179d,NaN,159.226.43.47,10
4,01a3324e-7ad9-4563-b598-0c2499068a39,LocalProvider,128.135.204.6,8


Example merging in the end:
```python
o = pd.merge(func_mapping,data, on='imports')
p = pd.merge(task_mapping,o, on='func4.name')
r = pd.merge(task, p, on='task_uuid')
```